## COMP47250 Team Software Project : University College Dublin
## Interim Demo VQC_iris / SVC UI Wrapper
Aldridge Abaasa, Neha Mutha, Rahul Matai, Tinoda Garapasi

In [ ]:
from flask import Flask, render_template, jsonify, Response
import numpy as np
import io 
import base64
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from qiskit.circuit.library import ZZFeatureMap, PauliFeatureMap, RealAmplitudes
from qiskit.algorithms.optimizers import SPSA
from qiskit_machine_learning.algorithms import VQC
from qiskit import Aer, execute
from qiskit.visualization import plot_histogram
import time
import random
from flask_executor import Executor
from azure.quantum import Workspace
from azure.quantum.qiskit import AzureQuantumProvider
from sklearn.decomposition import PCA
from sklearn import decomposition
app = Flask(__name__)


executor = Executor(app)

class QuantumJob:
    def __init__(self):
        self.var_circuit = None
        self.backend = None
        self.training_complete = False
        self.progress = 0

    def qsettings(self,var_circuit,backend):
        # ... your training code ...
        self.var_circuit = var_circuit
        self.backend = backend
        self.training_complete = True
       

    def run_quantum_job(self):
        if self.var_circuit is not None and self.backend is not None:
            params = self.var_circuit.parameters
            random_values = [random.uniform(0, 2 * np.pi) for _ in range(len(params))]
            bound_circuit = self.var_circuit.bind_parameters(dict(zip(params, random_values)))
            total_iterations = 10
            progress_values = []
            job = execute(bound_circuit, backend=self.backend)
            result = job.result()
            counts = result.get_counts()
            fig = plot_histogram(counts)
            fig.savefig('static/histogram.jpg')  
            plt.close(fig)  
            return result  # or return a result if needed
            
quantum_job = QuantumJob()          
@app.route('/')
def home():
    iris = datasets.load_iris()
    X = iris.data[:60]
    y = iris.target[:60]
    
    pca = decomposition.PCA(n_components=4)
    pca.fit(X)
    X = pca.transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    N = X.shape[1]
    
    
    
    S = X.shape[0]
    E = "Linear"
    name = "iris"
    O = "ADAM"
    M ="QSVC/SVC"
    fm = create_feature_map("zz",N)
    A = ansatz_cir(N)
    
    feature_map = ZZFeatureMap(feature_dimension=N, reps=1)
    ansatz = RealAmplitudes(num_qubits=N, reps=3)
    optimizer = SPSA(maxiter=40)

    objective_func_vals = []
    
    def callback_graph(weights, obj_func_eval):
        objective_func_vals.append(obj_func_eval)
        plt.title("Objective function value against iteration")
        plt.xlabel("Iteration")
        plt.ylabel("Objective function value")
        plt.plot(range(len(objective_func_vals)), objective_func_vals)
        plt.savefig('static/graph.jpg')  # Save the graph as an image
        plt.close()  # Close the plot to prevent showing in the console

    vqc = VQC(
        feature_map=feature_map,
        ansatz=ansatz,
        optimizer=optimizer,
        callback=callback_graph
    )
    
    # Start the training in a separate thread
    import threading
    def train():
        #vqc.fit(X_train_scaled, y_train)
        provider = AzureQuantumProvider (
            resource_id = "/subscriptions/e7eef170-5f0d-42c3-a6c6-9ae33096de85/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/Quantamunia",
            location = "West Europe")
        provider.backends()
        backend = provider.get_backend("ionq.simulator")
        print(backend)
        var_circuit = vqc.circuit
        quantum_job.qsettings(var_circuit, backend)
        #quantum_job.run_quantum_job()
        
    train_thread = threading.Thread(target=train)
    train_thread.start()
        
    

    return render_template('index.html', features=N, samples = S, entang = E, name=name, optimiser = O, model=M, fmap=fm, ans=A )

def create_feature_map(feature_map_type, feature_dimension):
    if feature_map_type == 'zz':
        image_stream = io.BytesIO()
        feature_map = ZZFeatureMap(feature_dimension=feature_dimension, reps=1)
        feature_map.decompose().draw(output="mpl", fold=20,filename=image_stream  )
        image_stream.seek(0)
        base64_image = base64.b64encode(image_stream.getvalue()).decode('utf-8')
        return base64_image
    
def ansatz_cir(N):
    image_stream = io.BytesIO()
    ansatz = RealAmplitudes(num_qubits=N, reps=3)
    ansatz.decompose().draw(output="mpl", fold=20,filename=image_stream )
    image_stream.seek(0)
    base64_image = base64.b64encode(image_stream.getvalue()).decode('utf-8')
    return base64_image
    
@app.route('/get_graph_data')
def get_graph_data():
    # Return the current timestamp as a unique parameter to prevent caching
    timestamp = int(time.time() * 1000)
    return jsonify({'graph_src': f'static/graph.jpg?{timestamp}'})

@app.route('/quantum_job')
def quantum_job_route():
    if quantum_job.training_complete:
        quantum_job.run_quantum_job() 
        quantum_job.training_complete = False
        
        return jsonify({'result': 'success', 'progress': 0})
    else:
        return jsonify({'result': 'training_in_progress', 'progress': quantum_job.progress})


@app.route('/get_hist')
def get_hist():
     # Return the current timestamp as a unique parameter to prevent caching
    timestamp = int(time.time() * 4000)
    return jsonify({'hist_src': f'static/histogram.jpg?{timestamp}'})

    
  
  

    

if __name__ == '__main__':
    app.run(port=5887, debug=False, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5887
Press CTRL+C to quit
127.0.0.1 - - [28/Jun/2023 12:37:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:37:57] "GET /static/graph.jpg HTTP/1.1" 304 -
127.0.0.1 - - [28/Jun/2023 12:37:57] "GET /static/histogram.jpg HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:37:57] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:37:57] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:37:57] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [28/Jun/2023 12:37:57] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:37:57] "GET /static/graph.jpg?1687952277684 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:37:57] "GET /static/histogram.jpg?6751809110861 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:37:57] "GET /static/histogram.jpg?6751809110921 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:37:59] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:37:59] "GET /static/graph.jpg?1687952279708 HTTP/1.1" 200 -
127.

ionq.simulator


127.0.0.1 - - [28/Jun/2023 12:38:09] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:38:09] "GET /static/graph.jpg?1687952289098 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:38:11] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:38:11] "GET /static/histogram.jpg?6751809164221 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:38:11] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:38:11] "GET /static/histogram.jpg?6751809164373 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:38:11] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:38:11] "GET /static/graph.jpg?1687952291113 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:38:13] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:38:13] "GET /static/graph.jpg?1687952293133 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:38:15] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:38:15] "GET /static/histogram.jpg?6751809180290 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/

127.0.0.1 - - [28/Jun/2023 12:38:59] "GET /static/histogram.jpg?6751809357587 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:38:59] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:38:59] "GET /static/graph.jpg?1687952339737 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:39:01] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:39:01] "GET /static/graph.jpg?1687952341756 HTTP/1.1" 200 -


...

127.0.0.1 - - [28/Jun/2023 12:57:40] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:57:40] "GET /static/graph.jpg?1687953460035 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:57:42] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:57:42] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:57:42] "GET /static/histogram.jpg?6751813848159 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:57:42] "GET /static/histogram.jpg?6751813848215 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:57:43] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:57:43] "GET /static/graph.jpg?1687953463037 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:57:46] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:57:46] "GET /static/graph.jpg?1687953466045 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:57:47] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:57:47] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:57:47] "GET /stati

127.0.0.1 - - [28/Jun/2023 12:58:46] "GET /static/histogram.jpg?6751814104166 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:58:48] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:58:48] "GET /static/graph.jpg?1687953528046 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:58:51] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:58:51] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:58:51] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:58:51] "GET /static/histogram.jpg?6751814124163 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:58:51] "GET /static/graph.jpg?1687953531043 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:58:51] "GET /static/histogram.jpg?6751814124195 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:58:54] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:58:54] "GET /static/graph.jpg?1687953534033 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:58:56] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/

127.0.0.1 - - [28/Jun/2023 12:59:56] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:59:56] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:59:56] "GET /static/graph.jpg?1687953596052 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:59:56] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:59:56] "GET /static/histogram.jpg?6751814384240 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:59:56] "GET /static/histogram.jpg?6751814384308 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:59:58] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 12:59:58] "GET /static/graph.jpg?1687953598070 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:00:01] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:00:01] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:00:01] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:00:01] "GET /static/histogram.jpg?6751814404157 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:00:01] "GET /s

127.0.0.1 - - [28/Jun/2023 13:00:59] "GET /static/histogram.jpg?6751814636279 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:00:59] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:00:59] "GET /static/graph.jpg?1687953659483 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:01:02] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:01:02] "GET /static/graph.jpg?1687953662062 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:01:04] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:01:04] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:01:04] "GET /static/histogram.jpg?6751814656163 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:01:04] "GET /static/histogram.jpg?6751814656187 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:01:05] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:01:05] "GET /static/graph.jpg?1687953665038 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:01:08] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [2

127.0.0.1 - - [28/Jun/2023 13:02:09] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:02:09] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:02:09] "GET /static/histogram.jpg?6751814916129 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:02:09] "GET /static/histogram.jpg?6751814916157 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:02:10] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:02:10] "GET /static/graph.jpg?1687953730040 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:02:13] "GET /get_graph_data HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:02:13] "GET /static/graph.jpg?1687953733032 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:02:14] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:02:14] "GET /get_hist HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:02:14] "GET /static/histogram.jpg?6751814936182 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 13:02:14] "GET /static/histogram.jpg?6751814936242 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/20